In [1]:
# step_001 - no control at all
# step_002 - stand up reward and parallel feet penalty
# step_003 - step_002 and forward reward
# step_004 - step_003 and control penalty, knee flex reward and foot up reward
# step_005 - stand up reward only

current_step = 'step_005'

In [2]:
!apt install swig cmake ffmpeg xvfb python3-opengl
!pip install pyvirtualdisplay imageio[ffmpeg]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.15).
Suggested packages:
  libgle3 python3-numpy python3-tk swig-doc swig-examples swig4.0-examples
  swig4.0-doc
The following NEW packages will be installed:
  freeglut3 libglu1-mesa python3-opengl swig swig4.0
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,940 kB of archives.
After this operation, 13.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 freeglut3 amd64 2.8.1-6 [74.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libglu1-mesa amd64 9.0.2-1 [145 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-opengl all 3.1.5+dfsg-1 [605 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/u

In [3]:
import os

NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

%env MUJOCO_GL=egl

from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1920, 1080))
virtual_display.start()

env: MUJOCO_GL=egl


In [4]:
# Prepare to load data from google drive
from google.colab import drive
import datetime

# CONNECT TO GOOGLE DRIVE
gdrive_path = '/content/drive'
drive.mount(gdrive_path)

# DEFINE WORK DIRECTORY
workDir = os.path.join(gdrive_path, 'My Drive', 'OP3_TRAINNING', current_step)
print('WorkDir:', workDir)

log_dir = os.path.join(workDir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
print('LogDir:', log_dir)

# create folder if it doesn't exists
if not os.path.exists(log_dir):
  os.makedirs(log_dir)

tf_log_dir = os.path.join(log_dir, 'tensorboard_logs')
print('TfLogDir:', tf_log_dir)

# create folder if it doesn't exists
if not os.path.exists(tf_log_dir):
  os.makedirs(tf_log_dir)

# best models
to_load_path = os.path.join(workDir, 'best')
print('to_load_path:', to_load_path)
algos = ["a2c", "ddpg", "ppo", "sac", "td3"]
for algo in algos:
  algo_load_path = os.path.join(to_load_path, algo)
  print('AlgoLoadPath:', algo_load_path)
  if not os.path.exists(algo_load_path):
    os.makedirs(algo_load_path)

Mounted at /content/drive
WorkDir: /content/drive/My Drive/OP3_TRAINNING/step_005
LogDir: /content/drive/My Drive/OP3_TRAINNING/step_005/20250912-233108
TfLogDir: /content/drive/My Drive/OP3_TRAINNING/step_005/20250912-233108/tensorboard_logs
to_load_path: /content/drive/My Drive/OP3_TRAINNING/step_005/best
AlgoLoadPath: /content/drive/My Drive/OP3_TRAINNING/step_005/best/a2c
AlgoLoadPath: /content/drive/My Drive/OP3_TRAINNING/step_005/best/ddpg
AlgoLoadPath: /content/drive/My Drive/OP3_TRAINNING/step_005/best/ppo
AlgoLoadPath: /content/drive/My Drive/OP3_TRAINNING/step_005/best/sac
AlgoLoadPath: /content/drive/My Drive/OP3_TRAINNING/step_005/best/td3


In [5]:
# clean content folder
import os
import shutil

# location
location = "/content"

# directories
dirs = ["sample_data"]

for dir in dirs:
    path = os.path.join(location, dir)
    try:
        shutil.rmtree(path)
    except OSError as e:
        print("Error: %s : %s" % (path, e.strerror))

In [6]:
# Install Darwin Model
model_path = '/content/op3'
%cd /

if os.path.isdir(model_path):
  print(f"The directory '{model_path}' exists - git pull")
  %cd {model_path}
  !git pull
else:
  print(f"The directory '{model_path}' does not exist - git clone")
  !git clone https://github.com/Gianzanti/op3_model.git {model_path}


/
The directory '/content/op3' does not exist - git clone
Cloning into '/content/op3'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 143 (delta 44), reused 132 (delta 33), pack-reused 0 (from 0)
Receiving objects: 100% (143/143), 14.62 MiB | 15.74 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [7]:
%cd /
!pip install -e {model_path}

/
Obtaining file:///content/op3
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.5/243.5 kB 26.6 MB/s eta 0:00:00
  Building editable for op3 (pyproject.toml) ... done
  Created wheel for op3: filename=op3-0.1.12-py3-none-any.whl size=1154 sha256=f24a5db7dcdd16a50d43c08d477dfc90550cca8d0f678b22ff99703d9b769e4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-nzv5p3s1/wheels/6e/41/34/9ebb5c3b64ee3d5f174ea5b433d00e8f3478f5497c112a46b1
Successfully built op3


In [8]:
# Install RL Zoo
trainner_path = '/content/rl-zoo'
%cd /

if os.path.isdir(trainner_path):
  print(f"The directory '{trainner_path}' exists - git pull")
  %cd {trainner_path}
  !git pull
else:
  print(f"The directory '{trainner_path}' does not exist - git clone")
  !git clone https://github.com/Gianzanti/rl-zoo.git {trainner_path}


/
The directory '/content/rl-zoo' does not exist - git clone
Cloning into '/content/rl-zoo'...
remote: Enumerating objects: 3683, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 3683 (delta 33), reused 19 (delta 19), pack-reused 3640 (from 3)
Receiving objects: 100% (3683/3683), 8.56 MiB | 4.61 MiB/s, done.
Resolving deltas: 100% (2249/2249), done.


In [9]:
%cd /
!pip install -e {trainner_path}

/
Obtaining file:///content/rl-zoo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 10.0 MB/s eta 0:00:00
  Building editable for rl_zoo3 (pyproject.toml) ... done
  Created wheel for rl_zoo3: filename=rl_zoo3-2.7.0-0.editable-py3-none-any.whl size=4428 sha256=bfae7ecc874fd8f208469436a3dae12ea6ea681070dee6f221f97d5a9a33b569
  Stored in directory: /tmp/pip-ephem-wheel-cache-vgdqiuo7/wheels/2d/0b/3c/be4c09b7d9d2a891b5d1bc1e086a8fe4f4b4f016a0613b625c
Successfully built rl_zoo3


In [10]:
%cd {trainner_path}

algos = {
  "a2c": {
    "lr": 7e-4, "dev": "cpu"
  },
  'ddpg': {
    'lr': 1e-3, 'dev': 'cuda'
  },
  'ppo': {
    'lr': 3e-4, 'dev': 'cpu'
  },
  'sac': {
    'lr': 3e-4, 'dev': 'cuda'
  },
  'td3': {
    'lr': 1e-3, 'dev': 'cuda'
  }
}

n_timestep = 3_000_000
# save_freq = 100_000
# eval_freq = 100_000
save_freq = min(100_000, int(n_timestep / 10))
eval_freq = min(200_000, int(n_timestep / 10))
max_episode_steps = 1000
wrapper = [{"gymnasium.wrappers.TimeLimit": {"max_episode_steps": max_episode_steps}}]
n_envs = 16
n_eval_envs = 3
eval_episodes = 30


# weights
keep_alive_weight = 1.0
control_weight = 0.00 #1e-3
target_distance = 5.5
velocity_weight = 0.00 #3.0
reach_target_reward = 20.0
knee_flex_weight = 0.00 #1e-3
feet_up_weight = 0.00 #1e-3
feet_misalign_weight = 0.00 #0.5


# keep_alive_weight: float = 1.0,
# control_weight: float = 1e-3,
# target_distance: float = 100.0,
# velocity_weight: float = 3.0,
# reach_target_reward: float = 100.0,
# knee_flex_weight: float = 1e-3,
# feet_up_weight: float = 1e-3,
# feet_misalign_weight: float = 0.05,

for algo, value in algos.items():
  print('Training:', algo)
  config = f'research_config/{algo}.yml'
  best = os.path.join(to_load_path, algo, 'best_model.zip')
  if not os.path.exists(best):
    best = ''
  else:
    best = f'-i {best}'

  train_cmd = f'python3 train.py --algo {algo} --env DarwinOp3-v3 -conf {config} \
-f "{log_dir}" --tensorboard-log "{tf_log_dir}" --save-freq {save_freq} \
--vec-env subproc --eval-freq {eval_freq} --n-eval-envs {n_eval_envs} --eval-episodes {eval_episodes} \
--env-kwargs keep_alive_weight:{keep_alive_weight} control_weight:{control_weight} \
target_distance:{target_distance} velocity_weight:{velocity_weight} \
reach_target_reward:{reach_target_reward} knee_flex_weight:{knee_flex_weight} \
feet_up_weight:{feet_up_weight} feet_misalign_weight:{feet_misalign_weight} \
--hyperparams n_envs:{n_envs} learning_rate:{value["lr"]} \
n_timesteps:{n_timestep} env_wrapper:"{wrapper}" --device {value["dev"]} {best} -P'

  print(train_cmd)
  !{train_cmd}

  video_cmd = f'python3 -m rl_zoo3.record_video --algo {algo} \
--env DarwinOp3-v3 -n 3000 --load-best -o "{log_dir}" -f "{log_dir}"'
  print(video_cmd)
  !{video_cmd}


Streaming output truncated to the last 5000 lines.
|    info_dst_org           | 0.144    |
|    info_feet_height       | 0.0514   |
|    info_feet_misalignment | 0.0834   |
|    info_knee_angvel       | 2.19     |
|    p_control              | 0        |
|    p_feet_misalignment    | 0        |
|    r_feet_up              | 0        |
|    r_forward              | 0        |
|    r_health               | 1        |
|    r_knee_flex            | 0        |
| rollout/                  |          |
|    ep_len_mean            | 216      |
|    ep_rew_mean            | 215      |
| time/                     |          |
|    episodes               | 32884    |
|    fps                    | 1049     |
|    time_elapsed           | 2647     |
|    total_timesteps        | 2778512  |
| train/                    |          |
|    actor_loss             | -2.77    |
|    critic_loss            | 0.0121   |
|    learning_rate          | 0.001    |
|    n_updates              | 173031   |
------